In [1]:
from EDTpy.ChipElements import *
import numpy as np
import copy
from EDTpy import settings
import EDTpy
from EDTpy import EmptyGeometry
from EDTpy import EmptyPath
from EDTpy.settings import *

# Installation of gdspy

In [2]:
%pip install gdspy

Note: you may need to restart the kernel to use updated packages.


# Testing if everything works

In [5]:
import gdspy

# First attempt to draw

In [53]:
class Marker(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        
        self + gdspy.Rectangle([-b/2-b/10, -b/2-b/10],
                               [b/2+b/10, b/2+b/10], layer = 0)
        
        mark = gdspy.boolean(gdspy.Rectangle([-b/2, -a/2],[b/2, a/2]), gdspy.Rectangle([-a/2, -b/2],
                               [a/2, b/2]), 'or', layer = 0)
        self - mark
        self.add_port([0, 0], 180)

In [8]:
sketch = EDTpy.EmptySketch()   # creating the layout
Marker = Marker()              # creating object
sketch.add_geometry(Marker)    # adding the object to layout
sketch.assemble()              # assemble
sketch.show()                  # plotting in a separate window

Assembling Empty Sketch: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


"'Empty Sketch', (0.0, 0.0), 0.0 deg, 0 ports\n"

# Creating and merging ports

In [20]:
class Port_with_C(EmptyGeometry):
    default_values = {
        "s": CPW.default_values['S'],
        "w": CPW.default_values['W'],
        "N": 2, #number of odd elements
        "l": 50, 
        "a": 3.3, # width of finger
        "b": 3.3, # gap between fingers
        "g": 10, # gap between fingers and GND
        "contact_pad_size": 200,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "CPW_C"
        
        w = values['w']
        s = values['s']
        N = int((values['N'])/2)
        a = values['a']
        b = values['b']
        l = values['l']+b*2
        g = values['g']
        gnd = 50
        
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']
        
        self + gdspy.Rectangle([0, -s/2-w],
                               [10, s/2+w], layer = 0)
        self - gdspy.Rectangle([0, -s/2],
                               [10, s/2], layer = 0)
        
        L = l+gnd
        jo = (N)*(a+b)*2+a/2
        jo_g = jo+g
        offset = L*0.4
        points = [(10, s/2+w), 
                  (offset, jo_g), (offset+L, jo_g), 
                  (offset+L, jo), (offset, jo), (10, s/2)]
        self + gdspy.Polygon(points)
        points = [(10, -s/2-w), 
                  (offset, -jo_g), (offset+L, -jo_g), 
                  (offset+L, -jo), (offset, -jo), (10, -s/2)]
        self + gdspy.Polygon(points)
        
        offset1 = offset*1.3
        self + gdspy.Rectangle([offset1, jo],
                               [offset1+l, -jo], layer = 0)
        
        self - gdspy.Rectangle([offset1,      a/2],
                               [offset1+l-b,  -a/2], layer = 0)
        
        for n in range(N):
            self - gdspy.Rectangle([offset1,     a/2+2*(a+b)*(n+1)-b],
                                   [offset1+l-b, a/2+2*(a+b)*(n+1)-b+a], layer = 0)
            self - gdspy.Rectangle([offset1,     -a/2-2*(a+b)*(n+1)+b],
                                   [offset1+l-b, -a/2-2*(a+b)*(n+1)+b-a], layer = 0)
            
            self - gdspy.Rectangle([offset1+b,     a/2+2*(a+b)*(n)+b],
                                   [offset1+l,     a/2+2*(a+b)*(n)+a+b], layer = 0)
            self - gdspy.Rectangle([offset1+b,    -a/2-2*(a+b)*(n)-b],
                                   [offset1+l,    -a/2-2*(a+b)*(n)-a-b], layer = 0)
        
        z = offset+L
        self + gdspy.Rectangle([z+100, -contact_pad_size/2-50],
                               [z+100+contact_pad_size+50, contact_pad_size/2+50], layer = 0)
        self - gdspy.Rectangle([z+100, -contact_pad_size/2],
                       [z+100+contact_pad_size, contact_pad_size/2], layer = 0)
        
        points = [(z, jo_g), 
                  (z+100, contact_pad_size/2+50), (z+100, contact_pad_size/2), 
                  (z, jo)]
        self + gdspy.Polygon(points)
        
        points = [(z, -jo_g), 
                  (z+100, -contact_pad_size/2-50), (z+100, -contact_pad_size/2), 
                  (z, -jo)]
        self + gdspy.Polygon(points)

        self.add_port([0, 0], 180)

In [29]:
sketch = EDTpy.EmptySketch()        

sketch.add_port([0, 0], 0)           #Marker port  --  0

Port_with_C_1 = Port_with_C()  
Port_with_C_1.merge_with(sketch.ports[0], 0)
sketch.add_geometry(Port_with_C_1)


sketch.add_port([0, 0], 180)           #Marker port  --  1

Port_with_C_2 = Port_with_C()          
Port_with_C_2.merge_with(sketch.ports[1], 0)
sketch.add_geometry(Port_with_C_2)

sketch.assemble()                    
sketch.show()                        

Assembling Empty Sketch: 100%|█████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2016.49it/s]


"'Empty Sketch', (0.0, 0.0), 0.0 deg, 2 ports\n"

# Filters

In [84]:
dc_s = 10
dc_w = 10

In [87]:
class Planar_inductor(EmptyGeometry):
    default_values = {
        "a": dc_s,
        "b": dc_w,
        "gap": dc_w,
        "N_sp": 12,
        "width": 200,
        "leg": 0,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Planar_inductor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        leg = values['leg']
        layer = values['layer']

        self + gdspy.Rectangle([0, w+b],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w-b], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b+b+a, a/2], layer = 0)
        
        self - gdspy.Rectangle([2*b+b, a/2],
                               [2*b+b+a, w+a/2-gap], layer = 0)

        for i in range(N_sp):
            self - gdspy.Rectangle([2*b+b+a+(2*a+2*b)*(i), w+a/2-gap],
                                   [2*b+b+a+b+(2*a+2*b)*(i), w-gap-a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i)+b, -w-a/2+gap+a], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+a+b+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)

        self + gdspy.Rectangle([2*b+b+(2*a+2*b)*(N_sp), a/2],
                               [2*b+b+a+(2*a+2*b)*(N_sp), w+a/2-gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        #leg for connection
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)  

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)
        
        
        
        
class Planar_capasitor(EmptyGeometry):
    default_values = {
        "a": dc_s,
        "b": dc_w,
        "gap": dc_w,
        "N_sp": 15,
        "width": 120,
        "layer": 0,
        "leg": 0
    }

    def _drawing(self, values):
        self.name = "Planar_capasitor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        layer = values['layer']
        leg = values['leg']

        self + gdspy.Rectangle([0, w],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b, a/2], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([2*b+(2*a+2*b)*(i), -a/2],
                                   [2*b+(2*a+2*b)*(i+1), a/2], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), w],
                                   [2*b+b+a+(2*a+2*b)*(i), a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), -w],
                                   [2*b+b+a+(2*a+2*b)*(i), -a/2-gap], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), -w+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -a/2], layer = 0)
            
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+2*b, a/2], layer = 0)
                    
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, w],
                               [(a+b)*2*(N_sp)+b*2+a+b, a/2+gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, -w],
                               [(a+b)*2*(N_sp)+b*2+a+b, -a/2-gap], layer = 0)
        
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)
        
        
        
class Contact_pad_spiral(EmptyGeometry):
    default_values = {
        "a": dc_s,
        "b": dc_w,
        "N_sp": 6,
        "contact_pad_size": 300,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Contact_pad_spiral"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']

        self + gdspy.Rectangle([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, (contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)],
                                 [-(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3), -(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)], layer = 0)

        
        self - gdspy.Rectangle([-contact_pad_size/2, -contact_pad_size/2],
                                 [contact_pad_size/2, contact_pad_size/2], layer = 0)

        self - gdspy.Rectangle([-a/2, contact_pad_size/2],
                                 [a/2, contact_pad_size/2+b], layer = 0)
        self - gdspy.Rectangle([-a/2, contact_pad_size/2+b],
                                 [contact_pad_size/2+b, contact_pad_size/2+b+a], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(i), contact_pad_size/2+b+a+(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [-contact_pad_size/2-b-a-(a+b)*(i), +contact_pad_size/2+b+(a+b)*(i+1)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-a-(a+b)*(i), contact_pad_size/2+b+(a+b)*(i+1)],
                                     [contact_pad_size/2+b+(a+b)*(i+1), contact_pad_size/2+b+a+(a+b)*(i+1)], layer = 0)

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), contact_pad_size/2+b+a+(a+b)*(N_sp)],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp), a/2], layer = 0)  

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), a/2],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, -a/2], layer = 0)  

        self.add_port([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, 0], 0)
        self.add_port([0, 0], 180)
        
        
class LCLC(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
        "w": 270,
        "N_sp": 10,
        "N_c1": 10,
        "N_ind": 10,
        "N_c2": 10,
        "line": 2,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        w=values['w']

        N_sp=values['N_sp']
        N_c1=values['N_c1']
        N_ind=values['N_ind']
        N_c2=values['N_c2']
        
        line=values['line']
        
        spiral = Contact_pad_spiral(N_sp=N_sp)
#         spiral.merge_with(self.ports[9], 1)
        self + spiral
        cap = Planar_capasitor(width=w, N_sp=N_c1, leg = 0)
        cap.merge_with(spiral.ports[0], 0)
        self + cap
        ind_oo = Planar_inductor(width=w, N_sp=N_ind, leg = 0)
        ind_oo.merge_with(cap.ports[1], 0)
        self + ind_oo
        cap_o1o = Planar_capasitor(width=w, N_sp = N_c2, leg = 0)
        cap_o1o.merge_with(ind_oo.ports[1], 0)
        self + cap_o1o
        x = cap_o1o.ports[1].position[0]
        y = cap_o1o.ports[1].position[1]
        self.add_port([0, 0], 0)
        self.add_port([x, y], 0)
        

In [88]:
sketch = EDTpy.EmptySketch()   
LCLC_1 = LCLC(N_sp=5, N_c1=15, N_ind=20, N_c2=15)   # setting individual parameters
sketch.add_geometry(LCLC_1)      
sketch.assemble()             
sketch.show()                 

Assembling Empty Sketch: 100%|██████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 500.16it/s]


"'Empty Sketch', (0.0, 0.0), 0.0 deg, 0 ports\n"

# CPW line

In [50]:
# setting cpw parameters
CPW.default_values['S'] = 18
CPW.default_values['W'] = 12
CPW.default_values['layer'] = 0

In [90]:
sketch = EDTpy.EmptySketch()
sketch + gdspy.Rectangle([-5000/2, -5000/2],
                         [5000/2, 5000/2], layer = 0)

sketch - gdspy.Rectangle([-5000/2 + 250, -5000/2 + 250],
                         [5000/2 - 250, 5000/2 - 250], layer = 0)



sketch.add_port([-2000, 2000], -180) #Marker port  --  0

CPW.default_values['S'] = 18
CPW.default_values['W'] = 12

marker = Marker()
marker.merge_with(sketch.ports[0], 0)
sketch.add_geometry(marker)
sketch.circular_array(marker)

sketch.add_port([-1650, 1200], 180) #Marker port  --  1

res_path = [[50, 1200], 
            [350, 1200], 
            [350, 1800], 
            [850, 1800], 
            [850, 1200], 
            [1300, 1200], 
            [1300, 1600], 
            [2000, 1600]]
res_R = 200
res = CPW(res_path, res_R)
print('Resonator length', res.length)
CPW_c_r = CPW_C(s=18, w=12)
CPW_c_r.merge_with(sketch.ports[1], 0)
sketch.add_geometry(CPW_c_r)
res.merge_with(CPW_c_r.ports[0], 0)
sketch.add_geometry(res)

x0 = 800
y0 = 400
text = gdspy.Text("R8.436", 250, (x0-145, y0+33))
sketch + text 


sketch.add_port([-1650, -1200], 0) #Marker port  --  2

dc_s = 30
dc_w = 30

lclc = LCLC(N_sp=4, N_c1=5, N_ind=5, N_c2=5, w=250, line=8)
lclc.merge_with(sketch.ports[2], 1)
sketch.add_geometry(lclc)

sketch.assemble()
sketch.save_gds("test")


Resonator length 3034.955592153876


Assembling Empty Sketch: 100%|█████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1400.03it/s]
